In [ ]:
# This notebook is NOT executable. It is only provided as part of the documentation for the analysis.
# Please make your copy and take care of the paths to your data and tools.

In [ ]:
# Supplementary Figure 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess as sp

In [ ]:
tad_path="path/to/tad"
output_path="path/output"
bed_path = "path/to/groups/bedfiles/"

In [ ]:
in_path = os.path.join(tad_path)
out_path = os.path.join(output_path)
bed_path = os.path.join(bed_path)
boundaries_file = os.path.join(in_path, "mergedSamples_wt_bs2kb.corrected_bonferroni_boundaries.bed")

In [ ]:
# Supplementary Figure 1B

In [ ]:
# Insualtion score of WT hic data on the TSS of different groups of genes

In [ ]:
cmd = "module load deeptools;"
cmd += "computeMatrix reference-point "
cmd += "-S "+in_path+"mergedSamples_wt_bs2kb.corrected_bonferroni_score.bw "
cmd += "-R "
cmd += bed_path+"Domino-study.grouping_figure3.TSS_uniq.group_h2av-.bed "
cmd += bed_path+"Domino-study.grouping_figure3.TSS_uniq.group_h2av+.bed "
cmd += bed_path+"Domino-study.grouping_figure3.TSS_uniq.group_zld-dep.bed "
cmd += "-a 100000 -b 100000 --referencePoint TSS -o "+out_path+"insulationScoreOnRegions.matrix.gz "
cmd += "--missingDataAsZero -p 20 -bs 2000 "
print(cmd)
sp.check_output(cmd, shell = True)

In [ ]:
cmd = "module load deeptools/3.4.3;"
cmd += "plotHeatmap -m "+out_path+"insulationScoreOnRegions.matrix.gz "
cmd += "-o "+out_path+"insulationScoreOnRegions.plotheatmap.pdf --samplesLabel 'wt insulation score' "
cmd += "--dpi 300 --colorMap RdBu  --whatToShow 'heatmap and colorbar' --regionsLabel h2av- h2av+ zld-dep "
cmd += "--zMin -0.3 --zMax 0.3 "
sp.check_output(cmd, shell = True)

In [ ]:
# Supplementary Figure 1C

In [ ]:
cmd = "module load deeptools/3.4.3;"
cmd += "plotProfile -m "+out_path+"insulationScoreOnRegions.matrix.gz "
cmd += "-o "+out_path+"insulationScoreOnRegions.plotProfile.pdf "
cmd += "--dpi 300 --legendLocation none --colors '#0F4C81' '#FA7A35' '#1D6960' "
cmd += "--refPointLabel TSS -z h2av- h2av+ zld-dep --samplesLabel 'wt insulation score' "
sp.check_output(cmd, shell = True)

In [ ]:
# Supplementary Figure 1D

In [ ]:
for file in os.listdir(bed_path):
    if file.endswith(".bed"):
        cmd = "module load bedtools2;"
        cmd += "bedtools sort -i "+bed_path+file+" | "
        cmd += "bedtools closest -k 1 -d -a stdin "
        cmd += " -b "+boundaries_file+" > "
        cmd += "boundaries_dist_to_"+file.split("_")[-1]
        sp.check_output(cmd, shell=True)

In [ ]:
h2av_pos = pd.read_csv("boundaries_dist_to_h2av+.bed", sep = "\t", header = None)
h2av_neg = pd.read_csv("boundaries_dist_to_h2av-.bed", sep = "\t", header = None)
zld_dep = pd.read_csv("boundaries_dist_to_zld-dep.bed", sep = "\t", header = None)
inactive = pd.read_csv("boundaries_dist_to_inactive.bed", sep = "\t", header = None)
inactive = inactive.loc[inactive[12] != -1 ]

In [ ]:
dist_h2av_pos = h2av_pos[12]
dist_h2av_neg = h2av_neg[12]
dist_zld_dep = zld_dep[12]
dist_inactive = inactive[12]

In [ ]:
labels=["inactive", "h2av_neg", "h2av_pos", "zld_dep"]
colors = ['#873600', '#0F4C81', '#FA7A35', '#1D6960']
data = [dist_inactive.values, dist_h2av_neg.values,
        dist_h2av_pos.values, dist_zld_dep.values]
fig, ax = plt.subplots()
parts = ax.violinplot(data, showextrema=False)
for ind, pc in enumerate(parts['bodies']):
    pc.set_facecolor(colors[ind])
    pc.set_edgecolor(colors[ind])
    pc.set_alpha(1)

whiskers = []
medians = np.array([])
q1s = np.array([])
q3s = np.array([])
for x in data:
    x = np.sort(x)
    q1, median, q3 = np.percentile(x, [25, 50, 75])
    print(q1, median, q3)
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, x[-1])
    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, x[0], q1)
    whiskers.append([lower_adjacent_value, upper_adjacent_value])
    medians = np.append(medians, median)
    q1s = np.append(q1s,q1)
    q3s = np.append(q3s,q3)
whiskers=np.asarray(whiskers)
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]
inds = np.arange(1, len(medians) + 1)
ax.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
ax.vlines(inds, q1s, q3s, color='k', linestyle='-', lw=5)
ax.vlines(inds, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
ax.set_ylabel('Distance')
ax.get_xaxis().set_tick_params(direction='out')
ax.xaxis.set_ticks_position('bottom')
ax.set_xticks(np.arange(1, 5))
ax.set_xticklabels(labels)
ax.set_xlim(0.25, len(labels) + 0.75)
plt.ylim((-1000,80000))
plt.savefig("violinplot_zoomedIn.pdf",dpi=300)